In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
import umap
from itertools import combinations
from pathlib import Path

In [183]:
# PARAMETROS
CSV_PATH = "trayectorias_educativas.csv"
SAMPLE_SIZE = 10000 
IQR_MULTIPLIER = 1.5 # Para detección de outliers univariada

In [184]:
df_raw = pd.read_csv(CSV_PATH)
df = df_raw.sample(n=SAMPLE_SIZE, random_state=42)
print(f"Datos cargados: {df.shape}")
df.head()

C:\Users\benja\AppData\Local\Temp\ipykernel_25252\2723425568.py:1: DtypeWarning: Columns (20,22,26,47,147,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(CSV_PATH)


Datos cargados: (10000, 151)


,MRUN,ANYO_PROCESO_x,CODIGO_CARRERA,NOMBRE_CARRERA,SEDE,SIGLA_UNIVERSIDAD,PREFERENCIA,PUNTAJE_PONDERADO,LUGAR_EN_LA_LISTA,VIA_INGRESO,...,CODIGO_PROVINCIA_DOMICILIO,CODIGO_COMUNA_DOMICILIO,NOMBRE_COMUNA_DOMICILIO,SEXO,FECHA_NACIMIENTO_y,INGRESO_PERCAPITA_GRUPO_FA,RAZON_PRINCIPAL_PAES,year,MATE1_INV_ANTERIOR,MATE2_INV_ANTERIOR
160188,4716599,2025,16005,INGENIERÍA CIVIL EN MECÁNICA,REGION METROPOLITANA,USACH,2,"817,15",9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,,
4386,22473476,2024,38083,ARQUITECTURA,SANTIAGO,UDP,3,"746,8",39,1,...,131.0,13112.0,LA PINTANA,1.0,200411.0,1.0,3,2024,NaN,NaN
11906,3255544,2024,50807,INGENIERÍA CIVIL INFORMÁTICA,SANTIAGO,USS,1,"696,75",47,1,...,133.0,13302.0,LAMPA,1.0,200512.0,5.0,3,2024,NaN,NaN
229490,17372710,2025,41168,OBSTETRICIA,SANTIAGO,UNAB,1,"710,2",29,1,...,131.0,13118.0,MACUL,2.0,200505.0,10.0,8,2025,,
116182,8350855,2024,42003,DERECHO,SANTIAGO,UAI,1,"789,5",64,1,...,131.0,13122.0,PEÑALOLEN,1.0,200505.0,99.0,3,2024,NaN,NaN


In [185]:
print("Dimensiones (filas, columnas):", df.shape)

Dimensiones (filas, columnas): (10000, 151)


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 160188 to 46978
Columns: 151 entries, MRUN to MATE2_INV_ANTERIOR
dtypes: float64(85), int64(7), object(59)
memory usage: 11.6+ MB


In [187]:
columnas = list(df.columns)

In [188]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

In [189]:
numeric_cols

['MRUN',
 'ANYO_PROCESO_x',
 'CODIGO_CARRERA',
 'PREFERENCIA',
 'LUGAR_EN_LA_LISTA',
 'VIA_INGRESO',
 'ANYO_PROCESO_y',
 'COD_SEXO',
 'FECHA_NACIMIENTO_x',
 'PTJE_NEM',
 'PORC_SUP_NOTAS',
 'PTJE_RANKING',
 'CLEC_MAX',
 'MATE1_MAX',
 'MATE2_MAX',
 'HCSOC_MAX',
 'CIEN_MAX',
 'RINDIO_PROCESO_ANTERIOR',
 'RINDIO_PROCESO_ACTUAL',
 'HABILITACION_POST',
 'FORMA_REG_CL',
 'CORRECTAS_REG_CL',
 'ERRADAS_REG_CL',
 'OMITIDAS_REG_CL',
 'FORMA_REG_M1',
 'CORRECTAS_REG_M1',
 'ERRADAS_REG_M1',
 'OMITIDAS_REG_M1',
 'FORMA_REG_M2',
 'CORRECTAS_REG_M2',
 'ERRADAS_REG_M2',
 'OMITIDAS_REG_M2',
 'FORMA_REG_HCS',
 'CORRECTAS_REG_HCS',
 'ERRADAS_REG_HCS',
 'OMITIDAS_REG_HCS',
 'FORMA_REG_CBIO',
 'CORRECTAS_REG_CBIO',
 'ERRADAS_REG_CBIO',
 'OMITIDAS_REG_CBIO',
 'FORMA_REG_CFIS',
 'CORRECTAS_REG_CFIS',
 'ERRADAS_REG_CFIS',
 'OMITIDAS_REG_CFIS',
 'FORMA_REG_CQUI',
 'CORRECTAS_REG_CQUI',
 'ERRADAS_REG_CQUI',
 'OMITIDAS_REG_CQUI',
 'FORMA_REG_CTP',
 'CORRECTAS_REG_CTP',
 'ERRADAS_REG_CTP',
 'OMITIDAS_REG_CTP',
 'F

In [190]:
columnas

['MRUN',
 'ANYO_PROCESO_x',
 'CODIGO_CARRERA',
 'NOMBRE_CARRERA',
 'SEDE',
 'SIGLA_UNIVERSIDAD',
 'PREFERENCIA',
 'PUNTAJE_PONDERADO',
 'LUGAR_EN_LA_LISTA',
 'VIA_INGRESO',
 'ANYO_PROCESO_y',
 'COD_SEXO',
 'FECHA_NACIMIENTO_x',
 'RBD',
 'CODIGO_ENS',
 'LOCAL_EDUCACIONAL',
 'UNIDAD_EDUCATIVA',
 'NOMBRE_UNIDAD_EDUC',
 'RAMA_EDUCACIONAL',
 'DEPENDENCIA',
 'CODIGO_REGION_EGRESO',
 'NOMBRE_REGION_EGRESO',
 'CODIGO_PROVINCIA_EGRESO',
 'NOMBRE_PROVINCIA_EGRESO',
 'CODIGO_COMUNA_EGRESO',
 'NOMBRE_COMUNA_EGRESO',
 'ANYO_DE_EGRESO',
 'PROMEDIO_NOTAS',
 'PTJE_NEM',
 'PORC_SUP_NOTAS',
 'PTJE_RANKING',
 'CLEC_REG_ACTUAL',
 'MATE1_REG_ACTUAL',
 'MATE2_REG_ACTUAL',
 'HCSOC_REG_ACTUAL',
 'CIEN_REG_ACTUAL',
 'CLEC_INV_ACTUAL',
 'MATE1_INV_ACTUAL',
 'MATE2_INV_ACTUAL',
 'HCSOC_INV_ACTUAL',
 'CIEN_INV_ACTUAL',
 'CLEC_REG_ANTERIOR',
 'MATE1_REG_ANTERIOR',
 'MATE2_REG_ANTERIOR',
 'HCSOC_REG_ANTERIOR',
 'CIEN_REG_ANTERIOR',
 'CLEC_INV_ANTERIOR',
 'MATE_INV_ANTERIOR',
 'HCSOC_INV_ANTERIOR',
 'CIEN_INV_ANTERI

# Limpieza de datos

datos nulos

In [191]:
df.isnull().sum()

MRUN                             0
ANYO_PROCESO_x                   0
CODIGO_CARRERA                   0
NOMBRE_CARRERA                   0
SEDE                             0
                              ... 
INGRESO_PERCAPITA_GRUPO_FA    3532
RAZON_PRINCIPAL_PAES          3532
year                             0
MATE1_INV_ANTERIOR            5115
MATE2_INV_ANTERIOR            5115
Length: 151, dtype: int64

In [192]:
df['NOMBRE_UNIDAD_EDUC'].isnull().sum()

np.int64(371)

In [193]:
cols_numericas = [
    'PUNTAJE_PONDERADO', 'PROMEDIO_NOTAS', 'PTJE_NEM', 'PORC_SUP_NOTAS', 'PTJE_RANKING',
    'CLEC_REG_ACTUAL', 'MATE1_REG_ACTUAL', 'MATE2_REG_ACTUAL',
    'HCSOC_REG_ACTUAL', 'CIEN_REG_ACTUAL', 'CLEC_INV_ACTUAL',
    'MATE1_INV_ACTUAL', 'MATE2_INV_ACTUAL', 'HCSOC_INV_ACTUAL',
    'CIEN_INV_ACTUAL', 'CLEC_REG_ANTERIOR', 'MATE1_REG_ANTERIOR',
    'MATE2_REG_ANTERIOR', 'HCSOC_REG_ANTERIOR', 'CIEN_REG_ANTERIOR',
    'CLEC_INV_ANTERIOR', 'MATE_INV_ANTERIOR', 'HCSOC_INV_ANTERIOR','CIEN_INV_ANTERIOR'
]

def limpiar_columna_numerica(serie: pd.Series):
    serie = (
        serie.astype(str)
        .str.strip()
        .str.replace(r"[^\d,.\-]", "", regex=True)   # quitar símbolos
        .str.replace(".", "", regex=False)           # quitar separador de miles
        .str.replace(",", ".", regex=False)          # coma a punto
    )
    return pd.to_numeric(serie, errors="coerce")

# Aplicar la limpieza solo a las columnas indicadas
for col in cols_numericas:
    if col in df.columns:
        df[col] = limpiar_columna_numerica(df[col])


# Normalizacion de los datos

In [194]:
df = pd.get_dummies(df, drop_first=True, dummy_na=True)
print(f"Columnas después de get_dummies: {df.shape[1]}")

# Asegurar que todas las columnas sean numéricas
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Estandarización (media 0, varianza 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)
print("Normalización lista")

Columnas después de get_dummies: 10781
Normalización lista


In [195]:
X_scaled = df.apply(lambda s: s.fillna(s.median()))

In [196]:
X_scaled.head()

,MRUN,ANYO_PROCESO_x,CODIGO_CARRERA,PREFERENCIA,PUNTAJE_PONDERADO,LUGAR_EN_LA_LISTA,VIA_INGRESO,ANYO_PROCESO_y,COD_SEXO,FECHA_NACIMIENTO_x,...,MATE2_INV_ANTERIOR_434,MATE2_INV_ANTERIOR_463,MATE2_INV_ANTERIOR_495,MATE2_INV_ANTERIOR_518,MATE2_INV_ANTERIOR_532,MATE2_INV_ANTERIOR_580,MATE2_INV_ANTERIOR_599,MATE2_INV_ANTERIOR_626,MATE2_INV_ANTERIOR_654,MATE2_INV_ANTERIOR_nan
160188,4716599,2025,16005,2,817.15,9,1,2025.0,2.0,200605.0,...,False,False,False,False,False,False,False,False,False,False
4386,22473476,2024,38083,3,746.80,39,1,2024.0,1.0,200411.0,...,False,False,False,False,False,False,False,False,False,True
11906,3255544,2024,50807,1,696.75,47,1,2024.0,1.0,200512.0,...,False,False,False,False,False,False,False,False,False,True
229490,17372710,2025,41168,1,710.20,29,1,2025.0,2.0,200505.0,...,False,False,False,False,False,False,False,False,False,False
116182,8350855,2024,42003,1,789.50,64,1,2024.0,1.0,200505.0,...,False,False,False,False,False,False,False,False,False,True


In [197]:
list(X_scaled.columns)

['MRUN',
 'ANYO_PROCESO_x',
 'CODIGO_CARRERA',
 'PREFERENCIA',
 'PUNTAJE_PONDERADO',
 'LUGAR_EN_LA_LISTA',
 'VIA_INGRESO',
 'ANYO_PROCESO_y',
 'COD_SEXO',
 'FECHA_NACIMIENTO_x',
 'PROMEDIO_NOTAS',
 'PTJE_NEM',
 'PORC_SUP_NOTAS',
 'PTJE_RANKING',
 'CLEC_REG_ACTUAL',
 'MATE1_REG_ACTUAL',
 'MATE2_REG_ACTUAL',
 'HCSOC_REG_ACTUAL',
 'CIEN_REG_ACTUAL',
 'CLEC_INV_ACTUAL',
 'MATE1_INV_ACTUAL',
 'MATE2_INV_ACTUAL',
 'HCSOC_INV_ACTUAL',
 'CIEN_INV_ACTUAL',
 'CLEC_REG_ANTERIOR',
 'MATE1_REG_ANTERIOR',
 'MATE2_REG_ANTERIOR',
 'HCSOC_REG_ANTERIOR',
 'CIEN_REG_ANTERIOR',
 'CLEC_INV_ANTERIOR',
 'MATE_INV_ANTERIOR',
 'HCSOC_INV_ANTERIOR',
 'CIEN_INV_ANTERIOR',
 'CLEC_MAX',
 'MATE1_MAX',
 'MATE2_MAX',
 'HCSOC_MAX',
 'CIEN_MAX',
 'RINDIO_PROCESO_ANTERIOR',
 'RINDIO_PROCESO_ACTUAL',
 'HABILITACION_POST',
 'FORMA_REG_CL',
 'CORRECTAS_REG_CL',
 'ERRADAS_REG_CL',
 'OMITIDAS_REG_CL',
 'FORMA_REG_M1',
 'CORRECTAS_REG_M1',
 'ERRADAS_REG_M1',
 'OMITIDAS_REG_M1',
 'FORMA_REG_M2',
 'CORRECTAS_REG_M2',
 'ERRADAS_

# Detección de outliers

In [198]:
col = ["PUNTAJE_PONDERADO", "PTJE_RANKING", "PTJE_NEM"]

Q1 = X_scaled[col].quantile(0.25)
Q3 = X_scaled[col].quantile(0.75)
IQR = Q3 - Q1
k = 1.5  # puedes ajustar si quieres más o menos permisivo

lower = Q1 - k * IQR
upper = Q3 + k * IQR

mask = pd.DataFrame(True, index=X_scaled.index, columns=col)
for c in col:
    mask[c] = (df[c] >= lower[c]) & (df[c] <= upper[c])


mask_final = mask.all(axis=1)
df_clean = X_scaled.loc[mask_final].copy()

print(f"Outliers eliminados según {col}: {len(df) - len(df_clean)} / {len(df)} "
      f"({100*(1-len(df_clean)/len(df))}%)")

Outliers eliminados según ['PUNTAJE_PONDERADO', 'PTJE_RANKING', 'PTJE_NEM']: 1058 / 10000 (10.58%)


# Correlacion

# Reducción dimensionalidad: PCA

# Visualización: UMAP